# Legal Document Search Project

Welcome to the Legal Document Search Project. This project aims to provide an efficient and effective way to search through a large corpus of legal documents. 

We use Natural Language Processing (NLP) techniques, specifically BM25 (Best Matching 25) algorithm, to index and rank documents based on a given query. 

The project also includes a simple user interface built with Streamlit, allowing users to input their search queries and view the top matching documents.

The project uses several Python libraries, including NLTK for text processing, Polars for data manipulation, and Rank-BM25 for search model generation and querying. 

The project is organized as a Python Notebook, with each cell representing a step in the data processing and search pipeline. 

Please follow along with the cells in the notebook to understand the workflow of the project, section headers and other markdown are provided to understand what is going on with the code.


# Install dependencies
You will likely need to restart your kernel after installation   
\**This does not need to be run twice.*

In [1]:
%pip install polars
%pip install nltk
%pip install rank-bm25
%pip install scikit-learn
%pip install streamlit
%pip install streamlit_jupyter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 5.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 5.6 MB/s eta 0:

# Initialize NLTK and DataFrame

In [3]:
import polars as pl
df = pl.read_ndjson('/Users/abhishekshah/Desktop/Legal project/archive/text.data.jsonl')
df = df.slice(int(-0.5 * len(df)))
df.write_parquet('legal.corpus.par')


In [4]:
import polars as pl
import nltk
from pathlib import Path

# Ensure NLTK data is downloaded
nltk.download('stopwords')
print('\'stopwords\' corpus downloaded')
nltk.download('punkt')
print('\'punkt\' tokenizer downloaded')


# DATA DIRECTORIES
corpus_dir = Path('legal.corpus.par').resolve()
tokens_dir = Path('legal.tokens.par').resolve()

# Load data
documents_df = pl.read_parquet(corpus_dir)
print('\nData loaded to DataFrame:')
print(documents_df)




[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhishekshah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abhishekshah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'stopwords' corpus downloaded
'punkt' tokenizer downloaded

Data loaded to DataFrame:
shape: (91_574, 13)
┌─────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id      ┆ name       ┆ name_abbr ┆ decision_ ┆ … ┆ reporter  ┆ court     ┆ jurisdict ┆ casebody  │
│ ---     ┆ ---        ┆ eviation  ┆ date      ┆   ┆ ---       ┆ ---       ┆ ion       ┆ ---       │
│ i64     ┆ str        ┆ ---       ┆ ---       ┆   ┆ struct[1] ┆ struct[5] ┆ ---       ┆ struct[2] │
│         ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆ struct[5] ┆           │
╞═════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 2422138 ┆ People of  ┆ People v. ┆ 1943-12-1 ┆ … ┆ {"Illinoi ┆ {8837,"Il ┆ {29,"ill" ┆ {"ok",{[{ │
│         ┆ the State  ┆ Kabana    ┆ 4         ┆   ┆ s         ┆ linois    ┆ ,"Ill."," ┆ "majority │
│         ┆ of         ┆           ┆           ┆   ┆ Appellate ┆ Appellate ┆ Illinois"

### Schema and example data from corpus

In [ ]:
documents_df.schema

OrderedDict([('id', Int64),
             ('name', String),
             ('name_abbreviation', String),
             ('decision_date', String),
             ('docket_number', String),
             ('first_page', String),
             ('last_page', String),
             ('citations', List(Struct({'type': String, 'cite': String}))),
             ('volume', Struct({'volume_number': String})),
             ('reporter', Struct({'full_name': String})),
             ('court',
              Struct({'id': Int64, 'name': String, 'name_abbreviation': String, 'jurisdiction_url': Null, 'slug': String})),
             ('jurisdiction',
              Struct({'id': Int64, 'slug': String, 'name': String, 'name_long': String, 'whitelisted': Boolean})),
             ('casebody',
              Struct({'status': String, 'data': Struct({'opinions': List(Struct({'type': String, 'text': String, 'author': String})), 'parties': List(String), 'attorneys': List(String), 'head_matter': String, 'judges': List(Null)})

In [ ]:
# # Print the first 5 rows
# documents_df.head()


shape: (5, 13)
┌─────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id      ┆ name       ┆ name_abbr ┆ decision_ ┆ … ┆ reporter  ┆ court     ┆ jurisdict ┆ casebody  │
│ ---     ┆ ---        ┆ eviation  ┆ date      ┆   ┆ ---       ┆ ---       ┆ ion       ┆ ---       │
│ i64     ┆ str        ┆ ---       ┆ ---       ┆   ┆ struct[1] ┆ struct[5] ┆ ---       ┆ struct[2] │
│         ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆ struct[5] ┆           │
╞═════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 2422138 ┆ People of  ┆ People v. ┆ 1943-12-1 ┆ … ┆ {"Illinoi ┆ {8837,"Il ┆ {29,"ill" ┆ {"ok",{[{ │
│         ┆ the State  ┆ Kabana    ┆ 4         ┆   ┆ s         ┆ linois    ┆ ,"Ill."," ┆ "majority │
│         ┆ of         ┆           ┆           ┆   ┆ Appellate ┆ Appellate ┆ Illinois" ┆ ","Opinio │
│         ┆ Illinois,… ┆           ┆           ┆   ┆ Court     ┆ Court"…   ┆ ,true…    ┆ n by      │
│         ┆            ┆           ┆           ┆   ┆ Repor…    ┆           ┆           ┆ …         │
│ 2422447 ┆ Jerry R.   ┆ Collins   ┆ 1943-12-1 ┆ … ┆ {"Illinoi ┆ {8837,"Il ┆ {29,"ill" ┆ {"ok",{[{ │
│         ┆ Collins,   ┆ v. City   ┆ 4         ┆   ┆ s         ┆ linois    ┆ ,"Ill."," ┆ "majority │
│         ┆ Minor, by  ┆ of        ┆           ┆   ┆ Appellate ┆ Appellate ┆ Illinois" ┆ ","Mr.    │
│         ┆ Howa…      ┆ Chicago   ┆           ┆   ┆ Court     ┆ Court"…   ┆ ,true…    ┆ Justice…  │
│         ┆            ┆           ┆           ┆   ┆ Repor…    ┆           ┆           ┆           │
│ 2423379 ┆ Alice      ┆ Oliver v. ┆ 1943-12-1 ┆ … ┆ {"Illinoi ┆ {8837,"Il ┆ {29,"ill" ┆ {"ok",{[{ │
│         ┆ Oliver,    ┆ Crook     ┆ 4         ┆   ┆ s         ┆ linois    ┆ ,"Ill."," ┆ "majority │
│         ┆ Appellee,  ┆           ┆           ┆   ┆ Appellate ┆ Appellate ┆ Illinois" ┆ ","Mr.    │
│         ┆ v. Isabe…  ┆           ┆           ┆   ┆ Court     ┆ Court"…   ┆ ,true…    ┆ Justice…  │
│         ┆            ┆           ┆           ┆   ┆ Repor…    ┆           ┆           ┆           │
│ 2423437 ┆ People of  ┆ People v. ┆ 1943-12-1 ┆ … ┆ {"Illinoi ┆ {8837,"Il ┆ {29,"ill" ┆ {"ok",{[{ │
│         ┆ the State  ┆ Kabana    ┆ 4         ┆   ┆ s         ┆ linois    ┆ ,"Ill."," ┆ "majority │
│         ┆ of         ┆           ┆           ┆   ┆ Appellate ┆ Appellate ┆ Illinois" ┆ ","Opinio │
│         ┆ Illinois,… ┆           ┆           ┆   ┆ Court     ┆ Court"…   ┆ ,true…    ┆ n by      │
│         ┆            ┆           ┆           ┆   ┆ Repor…    ┆           ┆           ┆ …         │
│ 2423531 ┆ Relaco     ┆ Relaco    ┆ 1943-12-1 ┆ … ┆ {"Illinoi ┆ {8837,"Il ┆ {29,"ill" ┆ {"ok",{[{ │
│         ┆ Rosin      ┆ Rosin     ┆ 4         ┆   ┆ s         ┆ linois    ┆ ,"Ill."," ┆ "majority │
│         ┆ Products   ┆ Products  ┆           ┆   ┆ Appellate ┆ Appellate ┆ Illinois" ┆ ","Opinio │
│         ┆ Company,   ┆ Co. v.    ┆           ┆   ┆ Court     ┆ Court"…   ┆ ,true…    ┆ n by      │
│         ┆ I…         ┆ Nat…      ┆           ┆   ┆ Repor…    ┆           ┆           ┆ …         │
└─────────┴────────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘

In [7]:
# Print the structure of the first non-null `casebody` entry
print(documents_df.filter(documents_df['casebody'].is_not_null())['casebody'].to_frame().unnest('casebody').unnest('data').drop('judges').head(1))


shape: (1, 5)
┌────────┬─────────────────────────┬───────────────────┬────────────────────────┬──────────────────┐
│ status ┆ opinions                ┆ parties           ┆ attorneys              ┆ head_matter      │
│ ---    ┆ ---                     ┆ ---               ┆ ---                    ┆ ---              │
│ str    ┆ list[struct[3]]         ┆ list[str]         ┆ list[str]              ┆ str              │
╞════════╪═════════════════════════╪═══════════════════╪════════════════════════╪══════════════════╡
│ ok     ┆ [{"majority","Opinion   ┆ ["People of the   ┆ ["George G. Rinier and ┆ Gen. No. 42,777. │
│        ┆ by                      ┆ State of Illinoi… ┆ Ray E. La…             ┆ People of the S… │
│        ┆ Justice…                ┆                   ┆                        ┆                  │
└────────┴─────────────────────────┴───────────────────┴────────────────────────┴──────────────────┘


# Process Text

Extract, tokenize, then initialize engine   
   
Please use this section to process all text for the model. If you have issues running this notebook all at once AFTER running this section:
1. Restart your kernel
2. Start running the next section

In [8]:
def extract_text_from_casebody(casebody):
    # Check if 'data' and 'opinions' keys exist and are not empty
    if casebody and 'data' in casebody and 'opinions' in casebody['data'] and casebody['data']['opinions']:
        # Concatenate all 'text' fields from each opinion into one string
        return ' '.join([opinion['text'] for opinion in casebody['data']['opinions']])
    return None

# Apply the function to the DataFrame
documents_df = documents_df.with_columns(
    documents_df['casebody'].map_elements(extract_text_from_casebody, return_dtype=pl.String).alias('case_text')
)

# Verify the first few entries of the extracted text
print(documents_df['case_text'].head())

documents_df = documents_df.select(['id', 'name', 'name_abbreviation', 'case_text']).drop_nulls()


shape: (10,)
Series: 'case_text' [str]
[
	"Opinion by
Jus…
	"Mr. Justice Sc…
	"Mr. Justice Sc…
	"Opinion by
Pre…
	"Opinion by
Jus…
	"Mr. Presiding …
	"Opinion by
Jus…
	"Mr. Justice Wo…
	"Mr. Justice Do…
	"Mr. Justice Do…
]


### Tokenize the documents and save the dictionary.  
   
_This one might take a while_

In [9]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm # Progress bar (need .auto for polars async)

def tokenize_wrapper(text: str):
    tokens = word_tokenize(text)
    return [word for word in tokens if word.lower() not in stopwords.words('english')]

# Tokenize the text and display progress bar
tokenized_text = [tokenize_wrapper(text[0]) for text in tqdm(documents_df.select(pl.col('case_text')).rows(), ncols=100, desc='Tokenizing text: ')]

# del documents_df  # Remove the original DataFrame from memory for now
# print('Tokenization complete')

# # Tokenize the text and display progress bar
# tokenized_text = [word_tokenize(text[0]) for text in tqdm(documents_df.select(pl.col('case_text')).rows(), ncols=100, desc='Tokenizing text: ')
#                     if text[0].lower() not in stopwords.words('english')] # Remove stopwords

del documents_df # Remove the original DataFrame from memory for now
print('Tokenization complete')

# Store the tokenized documents in new parquet file
print('\nSaving tokenized documents to new dataframe... ', end='')
new_df = pl.DataFrame({'tokens': tokenized_text})
print('done')
print('Saving tokenized documents to new parquet file...', end='')
new_df.write_parquet(tokens_dir)
print('done')

print(f'Tokenized documents saved to new parquet file: {tokens_dir}')

Tokenizing text: 100%|███████████████████████████████████████| 91414/91414 [11:26<00:00, 133.10it/s]


Tokenization complete

Saving tokenized documents to new dataframe... done
Saving tokenized documents to new parquet file...done
Tokenized documents saved to new parquet file: /Users/abhishekshah/Desktop/Legal project/archive/legal.tokens.par


# BM25 search model
If you have already generated the `legal.tokens.parquet` file, you can run this code with a restarted kernel if your memory needs to clean up.

In [10]:
from pathlib import Path
from rank_bm25 import BM25Okapi
import polars as pl
import numpy as np
from nltk.tokenize import word_tokenize

# Redefined for use after kernel restart
def extract_text_from_casebody(casebody):
    # Check if 'data' and 'opinions' keys exist and are not empty
    if casebody and 'data' in casebody and 'opinions' in casebody['data'] and casebody['data']['opinions']:
        # Concatenate all 'text' fields from each opinion into one string
        return ' '.join([opinion['text'] for opinion in casebody['data']['opinions']])
    return None

# Load the tokenized documents
tokenized_df = pl.read_parquet(Path('legal.tokens.par').resolve())
print('Tokenized documents loaded')
# Initialize the BM25 model
bm25 = BM25Okapi(tokenized_df['tokens'].to_numpy())
print('BM25 model loaded')
del tokenized_df # memory cleanup

# Reload the original documents
documents_df = pl.read_parquet(Path('legal.corpus.par').resolve())
print('Original documents loaded')
documents_df = ( documents_df.select(pl.col('id'), pl.col('name'), pl.col('name_abbreviation'))
                             .with_columns(documents_df['casebody'].map_elements(extract_text_from_casebody, return_dtype=pl.String).alias('case_text')) )
print('Original documents reformatted')

# Define a function to search documents using BM25
def bm25_search(query) -> tuple[pl.DataFrame, np.ndarray]:
    # Tokenize the query
    tokenized_query = word_tokenize(query.lower())
    # Retrieve scores
    doc_scores = bm25.get_scores(tokenized_query)
    # Sort documents by their score
    sorted_docs_idx = np.argsort(doc_scores)[::-1]
    # Optionally, return top N documents
    top_n_idx = sorted_docs_idx[:10]
    return documents_df[top_n_idx], doc_scores[top_n_idx]

print('BM25 search function defined')

Tokenized documents loaded
BM25 model loaded
Original documents loaded
Original documents reformatted
BM25 search function defined


## Examples

In [ ]:
# Example search
query = 'court case'
top_docs, top_scores = bm25_search(query)
#convert top docs to dataframe
top_docs = pl.DataFrame(top_docs)
# Print the top documents
top_docs.head()

id,name,name_abbreviation,case_text
i64,str,str,str
2601518,"""People of the …","""People ex rel.…","""MR PRESIDING J…"
3120984,"""THE PEOPLE OF …","""People v. Carp…","""Mr. JUSTICE LO…"
3224292,"""JACK KIRBY, Pl…","""Kirby v. Chica…","""Miss PRESIDING…"
2725140,"""The Department…","""Department of …","""Mr. Justice Sc…"
4265479,"""THE PEOPLE OF …","""People v. Dixo…","""JUSTICE SOUTH …"


: 

In [12]:
print(top_scores)

[3.51830151 3.51322885 3.48857781 3.48620416 3.4735774  3.47306372
 3.47195338 3.46409671 3.45961744 3.44974373]


In [ ]:
# Example search
query = "murder"
top_docs, top_scores = bm25_search(query)
#convert top docs to dataframe
top_docs = pl.DataFrame(top_docs)
# Print the top documents
top_docs.head()

id,name,name_abbreviation,case_text
i64,str,str,str
5298583,"""ALVIN D. WELCH…","""Welch v. Johns…","""JUSTICE HARTMA…"
894452,"""In re M.M. et …","""People v. Sylv…","""JUSTICE O’MALL…"
256313,"""THE PEOPLE OF …","""People v. Herr…","""JUSTICE REID d…"
637144,"""LEROY O’SHIELD…","""O’Shield v. La…","""JUSTICE GORDON…"
3599255,"""ANNA SKAGGS, P…","""Skaggs v. Seni…","""PRESIDING JUST…"


In [14]:
#show the case text of the top document
print(top_docs['case_text'][0])

JUSTICE HARTMAN
delivered the opinion of the court:
Plaintiffs appeal the circuit court’s grant of summary judgment in favor of defendant, David N. Johnson (Johnson), finding that his statement of economic interests, as amended, met all statutory requirements. The appeal raises as issues whether (1) the original omissions on Johnson’s statement of economic interests must result in ineligibility for, or forfeiture of, the mayoralty of the City of Harvey, (2) Johnson’s amended statement of economic interests relates back to the date on which he filed his original statement, and (3) evidence was improperly submitted and considered at the March 15, 1991, hearing. In the interests of timeliness, we have already ruled upon the issues presented in this appeal. As indicated in our order of reversal issued on April 19, 1991, our reasons were to be set forth in an opinion, which now follows.
The City of Harvey mayoral election was to take place on April 2, 1991. Johnson, the incumbent mayor, sou

# User interface

In [ ]:

import ipywidgets as widgets
from IPython.display import display
import textwrap

# Create a text input widget for the search query
query_input = widgets.Text(
    value='',
    placeholder='Type your query here',
    description='Query:',
    disabled=False
)

# Create a button to execute the search
search_button = widgets.Button(
        description='Search',
        disabled=False,
        button_style='',
        tooltip='Click to search',
        icon='search'
    )

# Create a function to handle the search button click event
def on_search_button_clicked(b):
    # Get the query from the input widget
    query = query_input.value
    # Perform the search
    top_docs, top_scores = bm25_search(query)
    for i, row in enumerate(top_docs.head(10).rows()):
        print(f"Document ID:{row[0]}")
        print(f"{row[1]}'\nScore: {top_scores[i]}")
        # Wrap the text exceeding 200 characters
        wrapped_text = textwrap.fill(row[3][:350], width=115)
        print(f"Case Text: {wrapped_text}...")  # Print the first 100 characters of the case text
        print("\n\n")

# Attach the event handler to the button
search_button.on_click(on_search_button_clicked)

# Display the input bar and button on the same line
box = widgets.HBox([query_input, search_button])

# Display the widgets
display(box)

In [ ]:
# jupyter nbconvert --to script nltksearchv1.ipynb